# High-level TF MNIST Example

In [1]:
import numpy as np
import os
import tensorflow as tf
from common.params import *
from common.utils import *

In [2]:
print(np.__version__)
print(tf.__version__)

1.13.1
1.2.1


In [3]:
def create_model():
    conv1 = tf.layers.conv2d(X, filters=20, kernel_size=5, strides=1, padding='valid')
    tanh1 = tf.tanh(conv1)
    pool1 = tf.layers.max_pooling2d(tanh1, pool_size=2, strides=2, padding='valid')
    conv2 = tf.layers.conv2d(pool1, filters=50, kernel_size=5, strides=1, padding='valid')
    tanh2 = tf.tanh(conv2)
    pool2 = tf.layers.max_pooling2d(tanh2, pool_size=2, strides=2, padding='valid')
    flatten = tf.reshape(pool2, shape=[-1, 50*4*4])
    fc1 = tf.layers.dense(flatten, 500, activation=tf.tanh)
    logits = tf.layers.dense(fc1, N_CLASSES, name='output')
    return logits

In [4]:
def init_model():
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.MomentumOptimizer(learning_rate=LR, momentum=MOMENTUM)
    training_op = optimizer.minimize(loss)
    return training_op

In [5]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = mnist_for_library(channel_first=False, one_hot=False)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.int32)

CPU times: user 216 ms, sys: 196 ms, total: 412 ms
Wall time: 468 ms


In [6]:
# Place-holders
X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y = tf.placeholder(tf.int32, shape=[None])

In [7]:
%%time
# Initialise model
logits = create_model()

CPU times: user 44 ms, sys: 4 ms, total: 48 ms
Wall time: 47.9 ms


In [8]:
%%time
model = init_model()
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
#correct = tf.nn.in_top_k(logits, y, 1)
#accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 180 ms, sys: 408 ms, total: 588 ms
Wall time: 593 ms


In [9]:
%%time
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE):
        sess.run(model, feed_dict={X: data, y: label})
    # Log
    #acc_train = sess.run(accuracy, feed_dict={X: data, y: label})
    #print(j, "Train accuracy:", acc_train)
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
CPU times: user 1min 5s, sys: 14.8 s, total: 1min 19s
Wall time: 1min 12s


In [10]:
%%time
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(logits,1)
    output = sess.run(pred, feed_dict={X: data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 3.08 s, sys: 140 ms, total: 3.22 s
Wall time: 3.17 s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.988982371795
